# Feature Engineering

Poniższy notebook sporządzono na podstawie kursu dostępnego pod adresem https://www.kaggle.com/learn/feature-engineering. 

**Zawiera on jedynie wybrane zagadnienia!**

Agenda:
1. Baseline Model (brak poniżej, więcej: https://www.kaggle.com/matleonard/baseline-model)
2. **Categorical Encodings**
3. **Feature Generation**
4. **Feature Selection**

## 2) CATEGORICAL ENCODINGS
Rozwinięcie wiadomości z rozdziału 3. Categorical Variables z kursu "Intermediate Machine Learning". Oprócz zawartych w nim metod radzenia sobie ze zmiennymi kategorycznymi istnieją jeszcze inne, np.

- count encoding
- target encoding
- CatBoost encoding

#### Count Encoding

Polega na zastąpieniu każdej z wartości kategorycznych liczbą jej wystąpień w zbiorze danych. Przykładowo, jeśli wartość "GB" pojawia się w zbiorze 10 razy, to każde wystąpienie "GB" zostanie zastąpione liczbą 10. 

Chcąc skorzystać z tej metody należy użyć biblioteki 'category_encoders' oraz metody 'CountEncoder', która działa z metodami '.fit' oraz '.transform' pochodzącymi z pakietu Scikit-Learn.

<img src="Images/img_10.jpg">

#### Target Encoding

Polega na zastąpieniu wartości kategorycznych średnią wartością obliczoną na podstawie poszczególnych wartości zmiennej przewidywanej (target) dla danej wartości zmiennej kategorycznej (inaczej, grupuje unikalne wartości zmiennej kategorycznej, oblicza dla każdej grupy średnią wartość targetu i wstawia w każdy wiersz należący do tej grupy).

Ta technika wykorzystuje zmienną docelową (target) do utworzenia nowej cechy (która ma zastąpić cechę "kategorczną"), dlatego wykorzystanie tej nowej cechy do treningu lub walidacji modelu byłoby w pewnym sensie **wyciekiem danych** (data leakage). Należy zatem zastosować "target encodings" jedynie dla zbioru treningowego.

Chcąc skorzystać z tej metody należy użyć biblioteki 'category_encoders' oraz metody 'TargetEncoder', która działa z metodami '.fit' oraz '.transform' pochodzącymi z pakietu Scikit-Learn.

<img src="Images/img_11.jpg">

#### CatBoost Encoding

Metoda ta jest podobna do "Target Encoding", ponieważ również opiera się na wartościach zmiennej przewidywanej (target), jednakże stosując "CatBoost Encoding" wartość prawdopodobieństwa jest obliczana jedynie na podstawie wierszy poprzedzających wiersz, dla którego chcemy obliczyć nową wartość zmiennej kategorycznej.

<img src="Images/img_12.jpg">

## 3) FEATURE GENERATION
Technika polegająca na ulepszaniu modelu poprzez tworzenie nowych cech na podstawie surowego zbioru danych. Przykładowo, pracując na zbiorze danych możemy utworzyć nowe kolumny (cechy), zawierające np. 
- liczbę projektów w poszczególnych tygodniach (cecha zostanie utworzona w oparciu o kolumnę zaw. daty)
- czas trwania każdego projektu
- czas jaki upłynął od ukończenia projektu
- zmodyfikowane wartości liczbowe (np. jeśli ich wartości mają b.duży zakres - patrz: przykład poniżej)

Przykład:
**Modyfikacja wartości liczbowych**
Dla przykładowego zbioru danych zakres wartości w kolumnie "Goal" mówi nam, że większość projektów ma "cele" niższe niż 5000 USD. Istnieje jednak sporo przypadków sięgających 100 000 USD. Nasz model może pracować lepiej, gdy cechy mają "ujednolicone" wartości (normally distributed) więc może pomocna może być ich modyfikacja. Najpowszechniejszymi przykładami przekształceń wartości numerycznych są "pierwiastek kwadratowy (square root) oraz "algorytm naturalny" (natural algorithm). Te przekształcenia mogą również być pomocne w przypadku występowania wartości odstających (outliers)

Rozkład wartości cechy "Goal" przez modyfikacją:

<img src="Images/img_13.jpg">

Przekształcenie z użyciem "square root":

<img src="Images/img_14.jpg">

Przekształcenie z użyciem "natural algorithm (log)":

<img src="Images/img_15.jpg">

UWAGA!

Transformacja z użyciem logarytmu naturalnego (log) jest bezużyteczna dla modeli opartych o Drzewa Decyzyjne. Będzie ona za to pomocna w przypadku modeli liniowych i sieci neuronowych!

## 4) FEATURE SELECTION

Podczas pracy z danymi bardzo często będziemy mieli do czynienia z sytuacją, że mamy setki (a nawet więcej) cech (kolumn) pochodzących z oryginalnego zbioru danych lub utworzonych przez nas w procesie obróbki (Encodings, Feature Generation etc.). Prowadzi to do dwóch podstawowych problemót:
1. Im więcej cech (kolumn), tym większe ryzyko przetrenowania modelu.
2. Im więcej cech (kolumn), tym więcej czasu potrzeba na wytrenowanie modelu i dostrojenie hiperparametrów.

W związku z powyższym konieczne jest wybranie najlepszych cech dla naszego modelu (cech, które zawierają najwięcej informacji z punktu widzenia naszego modelu i prognoz, które chcemy przeprowadzać).

### Metody wyboru cech

#### Wybór funkcji jednej zmiennej (ang. Univariate Feature Selection)

Najprostsze i najszybsze metody oparta są na jednowymiarowych testach statystycznych (ang. univariate statistical tests). Polega to na tym, że dla każdej cechy mierzymy w jakim stopniu zmienna przewidywana (target) zależy od tej cechy - do pomiaru tej zależności może posłużyć np. **test chi-kwadrat χ2** lub **analiza wariancji (ANOVA, ang. analysis of variance)**.

W pakiecie Scikit-Learn do wyboru cech służy moduł **"feature_selection"**. Funkcja **SelectKBest** zwraca _K_ najlepszych cech z punktu widzenia funkcji oceniającej, którą może być jedna z trzech funkcji:
- chi-kwadrat χ2, 
- ANOVA (F-value) 
- wzajemny wynik informacyjny (ang. mutual information score).

F-value mierzy liniową zależność między cechą, a zmienną przewidywaną (targetem). Oznacza to, że wynik może wystarczająco nie uwzględniać relacji pomiędzy badaną cechą a targetem jeśli jest ona nieliniowa (patrz przykład zmiennych zależnych, gdzie Y to wart. bezwzlg. z X - liniowej zależności nie ma, ale ogólnie zależność istnieje i jest nią właśnie wart. bezwzgl.).

Wzajemny wynik informacyjny (ang. mutual information score) jest nieparametryczny więc może uwzględniać zależność nieliniową!

Stosując metodę SelectKBest definiujemy liczbę _K_ cech, które chcemy zachować jako wynik zastosowania jednej z funkcji oceniających. Używając _.fit_transform(features, target)_ otrzymujemy tablicę zawierającą tylko wybrane (poprzez SelectKBest) cechy.

**PRZYKŁAD**

<img src="Images/img_16.jpg">

Niestety, w powyższym przekształceniu zrobiliśmy błąd - obliczyliśmy test statystyczny (F-value = f_classif) używając całego zbioru danych. Oznacza to, że informacje zawarte w zbiorze testowym i walidacyjnym mogą wpływać na cechy, które chcemy ustatecznie zachować do trenowania modelu (jest to źródło _wycieku danych_ ). W celu naprawy tego błędu należy dokonać wyboru cech jedynie na zbiorze treningowym (odpalić "SelectKBest" na zbiorze treningowym a nie na całym zbiorze danych!)

<img src="Images/img_17.jpg">

Należy zwrócić uwagę, że obecnie otrzymaliśmy tablicę zawierającą same wartości wyboranych cech jedynie dla zbioru treningowego (zastosowaliśmy "SelectKBest" jedynie na zbiorze treningowym a więc zbiór walidacyjny i testowy mają nadal ten sam, początkowy, zbiór cech). Aby usunąć zbędne cechy ze zbioru walidacyjnego i testowego musimy dowiedzieć się, które kolumny zostały zachowane jako wynik "SeleckKBest". W tym celu można użyć metody _.inverse_transform_ , która zwraca tablicę o kształcie oryginalnych danych.

<img src="Images/img_18.jpg">

Jako wynik otrzymaliśmy DataFrame o tym samym indeksie i kolumnach, co początkowy zbiór treningowy, ale wszystkie usunięte za pomocą "SelectKBest" kolumny są wypełnione zerami. 

Teraz możemy wybrać tylko te kolumny, dla których wartości są różne od zera (variance != 0):

<img src="Images/img_19.jpg">

#### Regularyzacja L1

Metody jednowymiarowe (univariate feature selection - jw) uwzględniają tylko jedną cechę naraz podczas podejmowania decyzji. Zamiast tego możemy dokonać wyboru przy użyciu wszystkich cech, włączając je do modelu liniowego poprzez zastosowanie **regularyzacji L1**. W przeciwieństwie do regularyzacji L2 ( **grzbietowa** ), która oblicza kwadrat współczynników dla poszczególnych cech, regularyzacja L1 (zwana również **_Lasso_** ) oblicza wartość bezwzględną tych współczynników.

Wraz ze wzrostem "siły" regularyzacji, cechy które są mniej ważne z punktu widzenia zmiennej przewidywanej, są ustawiane na 0. Pozwala to dokonać wyboru cech poprzez dostosowanie parametru regularyzacji. Parametr ten możemy wybrać w taki sposób, aby jakość, dla cech które wybierzemy dla naszego modelu, była jak największa lub możemy wcześniej podjąć decyzje ile ostatecznie cech (kolumn) chcemy zachować.

Chcąc dokonać klasyfikacji z użyciem regresji możemy skorzystać z metod pakietu Scikit-Learn i są to **sklearn.linear_model.Lasso** lub **sklearn.linear_model.Logisticregression**. Obu tych metod możemy użyć wraz z metodą **sklearn.feature_selection.SelectFromModel** , która służy właśnie do wyboru cech (kolumn), dla których wartość współczynnika regularyzacji jest różna od zera. W przeciwnym wypadku, kod przypomina ten dla metod jednowymiarowych.

<img src="Images/img_20.jpg">

Podobnie, jak w przypadku metod jednowymiarowych, otrzymaliśmy tablicę wartości cech, które zostały zachowane i ponownie musimy dokonać konwersji do DataFrame, gdzie oprócz wartości będziemy mieli również nazwy cech (kolumn).

<img src="Images/img_21.jpg">

W powyższym przykładzie zastosowanie parametru C = 1 powoduje, że odrzucamy kolumnę _time_since_last_project_ .

**PODSUMOWANIE**

Ogólnie, metoda wyboru cech z użyciem regularyzacji L1 jest lepsza, aniżeli metody jednowymiarowe, ale potrafi być ona bardzo wolna, gdy posiadamy zbiór danych o dużej liczbie cech. Metody jednowymiarowe będą znacznie szybsze w przypadku dużych zbiorów danych, ale modele utworzone za ich pomocą nie są z kolei tak wydajne.